In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Set the page numbers
from_page = 1
to_page = 84

In [4]:
# Initialize lists
review_titles = []
#review_dates_original = []
review_dates = []
review_ratings = []
review_texts = []
page_number = []

for i in range(from_page, to_page + 1):
    response = requests.get(f"https://www.trustpilot.com/review/bluemovement.com?page={i}")
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")

    for review in soup.find_all(class_ = "paper_paper__1PY90 paper_square__lJX8a card_card__lQWDv card_noPadding__D8PcU styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"):
        # Review titles
        review_title = review.find(class_ = "typography_typography__QgicV typography_h4__E971J typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3 styles_reviewTitle__04VGJ")
        review_titles.append(review_title.getText())

        # Review dates
        #review_date_original = review.select_one(selector = "time")
        #review_dates_original.append(review_date_original.getText())

        # Convert review date texts into Python datetime objects
        review_date = review.select_one(selector="time").getText().replace("Updated ", "")
        if "hours ago" in review_date.lower() or "hour ago" in review_date.lower():
            review_date = dt.datetime.now().date()
        elif "a day ago" in review_date.lower():
            review_date = dt.datetime.now().date() - dt.timedelta(days=1)
        elif "days ago" in review_date.lower():
            review_date = dt.datetime.now().date() - dt.timedelta(days=int(review_date[0]))
        else:
            review_date = dt.datetime.strptime(review_date, "%b %d, %Y").date()
        review_dates.append(review_date)

        # Review ratings
        review_rating = review.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild()
        review_ratings.append(review_rating["alt"])
        
        # When there is no review text, append "" instead of skipping so that data remains in sequence with other review data e.g. review_title
        review_text = review.find(class_ = "typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3")
        if review_text == None:
            review_texts.append("")
        else:
            review_texts.append(review_text.getText())
        
        # Trustpilot page number
        page_number.append(i)

In [5]:
# Generate final dataframe from lists
df_reviews = pd.DataFrame(list(zip(review_titles, review_dates, review_ratings, review_texts, page_number)),
                          columns = ['review_title', 'review_date', 'review_rating', 'review_text', 'page_number'])

In [6]:
# Adjusting 'rating' column
rating = []
for i in range(len(df_reviews)):
    rating.append(df_reviews['review_rating'][i][6])

df_reviews['rating'] = rating
df_reviews['rating'] = df_reviews['rating'].astype(int)

df_reviews = df_reviews.drop('review_rating', 1)

In [7]:
df_reviews.head()

,review_title,review_date,review_text,page_number,rating
0,It was a breeze booking the Washing…,2022-06-11,It was a breeze booking the Washing machine fr...,1,4
1,Alles is subliem afgeleverd .Perfect,2022-05-28,Alles is subliem afgeleverd .,1,5
2,Very bad company,2022-05-27,Very bad company. Paying way too much for even...,1,1
3,I really enjoyed the service,2022-05-18,"I really enjoyed the service, quite fast and p...",1,5
4,Very fast service,2022-05-13,"Very fast service , the drivers were very fast...",1,5
